In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, GPT2Tokenizer, GPT2LMHeadModel
import requests

# CLIP 모델 및 프로세서 로드
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# GPT-2 모델 및 토크나이저 로드
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 이미지 로드 및 전처리
def load_image(image_path=None, url=None):
    if url:
        # URL에서 이미지 로드
        image = Image.open(requests.get(url, stream=True).raw)
    elif image_path:
        # 로컬 경로에서 이미지 로드
        image = Image.open(image_path)
    else:
        raise ValueError("Image path or URL must be provided.")
    return image

# 이미지에서 특징 추출
def extract_image_features(image_path=None, url=None):
    image = load_image(image_path=image_path, url=url)
    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# 이미지 특징과 가장 유사한 텍스트 생성
def find_similar_text(image_features, candidate_texts):
    inputs = clip_processor(text=candidate_texts, return_tensors="pt", padding=True)

    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)

    # 이미지 특징과 텍스트 특징 간의 유사도를 계산
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    similarity = torch.matmul(image_features, text_features.T)

    # 가장 유사한 텍스트를 선택
    best_match_index = similarity.argmax().item()
    return candidate_texts[best_match_index]

# 이미지 특징을 기반으로 캡션 생성
def generate_caption_from_image(image_path=None, url=None):
    # 이미지에서 특징 추출
    image_features = extract_image_features(image_path=image_path, url=url)

    # CLIP 모델로부터 텍스트 후보군 생성 (캡션 후보)
    candidate_texts = [
        "A photo of a cat",
        "A picture of a dog",
        "A landscape with mountains",
        "A person riding a horse",
        "An aerial view of a city",
        "A close-up shot of food",
    ]

    # 이미지 특징과 가장 유사한 텍스트 찾기
    best_caption = find_similar_text(image_features, candidate_texts)

    # GPT-2 모델을 사용하여 이미지 특징 기반으로 캡션 확장
    input_ids = gpt2_tokenizer.encode(best_caption, return_tensors='pt')

    with torch.no_grad():
        outputs = gpt2_model.generate(input_ids, max_length=30, num_return_sequences=1,
                                       num_beams=5, early_stopping=True)

    generated_caption = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_caption

# 예시 이미지 경로 또는 URL
image_path = "path/to/your/image.jpg"  # 로컬 이미지 경로
url = "http://images.cocodataset.org/val2017/000000039769.jpg"  # URL 이미지

# 캡션 생성
generated_caption = generate_caption_from_image(url=url)  # 또는 image_path=image_path
print("Generated Caption:", generated_caption)